In [1]:
# use seagen env
!pip show openai | grep Version
!pip show requests | grep Version

Version: 1.5.0
Version: 2.31.0


In [2]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()

def get_drug_info(query, limit):
    # Replace with your actual API key
    api_key = os.getenv('OPENFDA_API_KEY')
    
    # Base URL for the OpenFDA Drug Endpoint (modify as needed)
    base_url = 'https://api.fda.gov/drug/event.json'
    
    # Construct the full URL with the query and API key
    # Adjust parameters as necessary for your specific query needs
    request_url = f'{base_url}?search={query}&limit={limit}&api_key={api_key}'
    print(request_url)
    
    # Make the HTTP request to OpenFDA
    response = requests.get(request_url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Process and return the response
        return response.json()
    else:
        # Handle errors (e.g., network issues, bad request, exceeded rate limits)
        return f'Error: {response.status_code} - {response.text}'

# Example usage:
query = 'patient.drug.drugindication:prostate cancer&sort=receivedate:desc'
results = get_drug_info(query, 10)



https://api.fda.gov/drug/event.json?search=patient.drug.drugindication:prostate cancer&sort=receivedate:desc&limit=10&api_key=GbBobnJXjXP1YFhXZ051a6ZvrN2X7tQc45NBB782


In [3]:
# write results to data/dataset.json file
import json
with open('data/dataset.json', 'w') as outfile:
    json.dump(results, outfile)

In [4]:
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [6]:
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# create assistant
assistant = client.beta.assistants.create(
    name="Clinical Data Assistant 2",
    instructions="You are a personal Clinical Data Assistant. Answer only using the file(s) provided.",
    model="gpt-4-1106-preview",
)


In [13]:
ASSISTANT_ID = assistant.id

# Upload the file
file = client.files.create(
    file=open(
        "data/dataset.json",
        "rb",
    ),
    purpose="assistants",
)

# Update Assistant
assistant = client.beta.assistants.update(
    ASSISTANT_ID,
    tools=[{"type": "retrieval"}, {"type": "code_interpreter"}],
    file_ids=[file.id],
)
show_json(assistant)

{'id': 'asst_LgqBpOuP5pros8gCqVy23H94',
 'created_at': 1703627476,
 'description': None,
 'file_ids': ['file-oqqRl21JFPO9u8s0xNOUqXfh'],
 'instructions': 'You are a personal Clinical Data Assistant. Answer only using the file(s) provided.',
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'name': 'Clinical Data Assistant 2',
 'object': 'assistant',
 'tools': [{'type': 'retrieval'}, {'type': 'code_interpreter'}]}

In [14]:
import time

def submit_message(assistant_id, thread, user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )


def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id, order="asc")

def create_thread_and_run(user_input):
    thread = client.beta.threads.create()
    run = submit_message(ASSISTANT_ID, thread, user_input)
    return thread, run


# Pretty printing helper
def pretty_print(messages):
    print("# Messages")
    for m in messages:
        print(f"{m.role}: {m.content[0].text.value}")
    print()


# Waiting in a loop
def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run


In [15]:
thread, run = create_thread_and_run(
    """What are the most effective treatments for prostate cancer?"""
)


In [16]:
# Wait for Run
run = wait_on_run(run, thread)

pretty_print(get_response(thread)) # for regular response

# Messages
user: What are the most effective treatments for prostate cancer?



In [17]:
get_response(thread)

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_vUOWef3SlqVrFP3dKrujp0lA', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='What are the most effective treatments for prostate cancer?'), type='text')], created_at=1703627648, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_087oyp69ICF869BXgrG9cF8a')], object='list', first_id='msg_vUOWef3SlqVrFP3dKrujp0lA', last_id='msg_vUOWef3SlqVrFP3dKrujp0lA', has_more=False)